In [ ]:
pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 32.4 MB/s eta 0:00:00


In [ ]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357274 sha256=cd8f91f7ebee1745284356258fe6f370f984a2354b3c488ef85993c058452a94
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 11.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
import pickle

from itertools import chain
from implicit.bpr import BayesianPersonalizedRanking

from numpy import asarray
from numpy import savetxt

from scipy import sparse
from sklearn.model_selection import train_test_split

from surprise import Dataset
from surprise import Reader
from surprise import SVD, SVDpp, NMF

from surprise.model_selection import cross_validate
from surprise.dump import dump

from re import U
from bs4 import BeautifulSoup
from pymongo.operations import ReplaceOne
import requests

import nest_asyncio
import asyncio
from aiohttp import ClientSession

nest_asyncio.apply()

import pymongo
from pymongo import UpdateOne, ReplaceOne
from pymongo.errors import BulkWriteError

import datetime

from pprint import pprint

import os

import pandas as pd
import matplotlib.pyplot as plt
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import time
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from gensim import corpora
from gensim.models import TfidfModel
import gensim
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def mean_reciprocal_rank(recs, test):
    """
    Compute the Mean Reciprocal Rank (MRR).

    Parameters:
        recs (list): List of tuples (movie_id, score) in ranked order.
        test (list): List of tuples (movie_id, rating).

    Returns:
        float: Mean Reciprocal Rank (MRR).
    """
    test_ids = {movie_id for movie_id, _ in test}
    for rank, (movie_id, _) in enumerate(recs, start=1):
        if movie_id in test_ids:
            return 1 / rank
    return 0  # No relevant items found in the recommendations

def ndcg(recs, test):
    """
    Compute the Normalized Discounted Cumulative Gain (nDCG).

    Parameters:
        recs (list): List of tuples (movie_id, score) in ranked order.
        test (list): List of tuples (movie_id, rating).

    Returns:
        float: Normalized Discounted Cumulative Gain (nDCG).
    """
    test_ratings = dict(test)

    # Compute DCG
    dcg = sum(test_ratings.get(movie_id, 0) / np.log2(rank + 1) for rank, (movie_id, _) in enumerate(recs, start=1))

    # Compute Ideal DCG
    ideal_ranking = sorted(test_ratings.items(), key=lambda x: x[1], reverse=True)
    idcg = sum(rating / np.log2(rank + 1) for rank, (movie_id, rating) in enumerate(ideal_ranking, start=1))

    return dcg / idcg if idcg > 0 else 0

def precision_at_k(recs, test, k=50):
    """
    Compute Precision at k (P@k).

    Parameters:
        recs (list): List of tuples (movie_id, score) in ranked order.
        test (list): List of tuples (movie_id, rating).
        k (int): The number of top recommendations to consider.

    Returns:
        float: Precision at k.
    """
    test_ids = {movie_id for movie_id, _ in test}
    top_k_recs = recs[:k]
    hits = len([movie_id for movie_id, _ in top_k_recs if movie_id in test_ids])
    return hits / k if k > 0 else 0

def compute_metrics(recs, test, k=10):
    """
    Compute evaluation metrics.

    Parameters:
        recs (list): List of tuples (movie_id, score) in ranked order.
        test (list): List of tuples (movie_id, rating).
        k (int): The number of top recommendations to consider for P@k.

    Returns:
        dict: Dictionary of computed metrics.
    """
    metrics = {}
    metrics['precision_at_k'] = precision_at_k(recs, test, k=k)
    metrics['mean_reciprocal_rank'] = mean_reciprocal_rank(recs, test)
    metrics['ndcg'] = ndcg(recs, test)
    return metrics

In [ ]:
# Movies from zip file
url = "https://raw.githubusercontent.com/sierramk1/Letterboxd-Recommendation-System/main/movie_data.csv.zip"
movies = pd.read_csv(url, compression='zip')

# Users from csv - converting to raw URL
url = "https://raw.githubusercontent.com/sierramk1/Letterboxd-Recommendation-System/main/users_export.csv"
users = pd.read_csv(url)

url = "https://raw.githubusercontent.com/sierramk1/Letterboxd-Recommendation-System/main/training_data.csv"
ratings = pd.read_csv(url, nrows=1000000)


# Ratings from local file
ratings = ratings[['user_id', 'movie_id', 'rating_val']]

In [ ]:
def create_movie_data_sample(movies, movie_list):
    movie_df = movies[movies["movie_id"].isin(movie_list)].copy()
    movie_df = movie_df[["movie_id", "image_url", "movie_title", "year_released"]]
    movie_df["image_url"] = (
        movie_df["image_url"]
        .fillna("")
        .str.replace("https://a.ltrbxd.com/resized/", "", regex=False)
    )
    movie_df["image_url"] = (
        movie_df["image_url"]
        .fillna("")
        .str.replace(
            "https://s.ltrbxd.com/static/img/empty-poster-230.c6baa486.png",
            "",
            regex=False,
        )
    )

    return movie_df

# Generate training data sample
training_df = ratings.copy()

review_counts_df = (
    ratings.groupby("movie_id")
    .size()  # Count reviews per movie
    .reset_index(name="count")  # Rename the count column
)

threshold_movie_list = review_counts_df["movie_id"].to_list()

# Generate movie data CSV
movie_df = create_movie_data_sample(movies, threshold_movie_list)

# Use movie_df to remove any items from threshold_list that do not have a "year_released"
# This virtually always means it's a collection of more popular movies (such as the LOTR trilogy) and we don't want it included in recs
retain_list = movie_df.loc[
    (movie_df["year_released"].notna() & movie_df["year_released"] != 0.0)
]["movie_id"].to_list()

threshold_movie_list = [x for x in threshold_movie_list if x in retain_list]

In [ ]:
async def fetch(url, session, input_data={}):
    async with session.get(url) as response:
        try:
            return await response.read(), input_data
        except:
            return None, None

async def generate_ratings_operations(response, send_to_db=True, return_unrated=False):
    # Parse ratings page response for each rating/review, use lxml parser for speed
    soup = BeautifulSoup(response[0], "lxml")
    reviews = soup.findAll("li", attrs={"class": "poster-container"})

    # Create empty array to store list of bulk operations or rating objects
    ratings_operations = []
    movie_operations = []

    for review in reviews:
        movie_id = review.find("div", attrs={"class", "film-poster"})[
            "data-target-link"
        ].split("/")[-2]

        # Check for rating
        rating = review.find("span", attrs={"class": "rating"})
        if not rating:
            if not return_unrated:
                continue
            rating_val = -1
        else:
            rating_class = rating["class"][-1]
            rating_val = int(rating_class.split("-")[-1])

        rating_object = {
            "movie_id": movie_id,
            "rating_val": rating_val,
            "user_id": response[1]["username"],
        }

        if not send_to_db:
            ratings_operations.append(rating_object)
        else:
            # Add UpdateOne operations for database insertion
            ratings_operations.append(
                UpdateOne(
                    {"user_id": response[1]["username"], "movie_id": movie_id},
                    {"$set": rating_object},
                    upsert=True,
                )
            )
            movie_operations.append(
                UpdateOne(
                    {"movie_id": movie_id},
                    {"$set": {"movie_id": movie_id}},
                    upsert=True,
                )
            )

    return ratings_operations, movie_operations

def build_model(df, user_data, model='SVD', num_factors=10, learning_rate=0.01, regularization=0.05, iterations=100):
    import random
    import numpy as np
    from scipy.sparse import csr_matrix
    from implicit.bpr import BayesianPersonalizedRanking

    # Set random seed
    random.seed(12)
    np.random.seed(12)

    # Filter user_data based on the model type
    if model == 'BPR':
        # Include both rated items and likes for BPR
        user_data_filtered = user_data
    else:
        # Exclude likes for SVD and NMF
        user_data_filtered = [x for x in user_data if x['rating_val'] > 0]

    # Convert filtered user_data to a DataFrame and append it to the existing data
    user_df = pd.DataFrame(user_data_filtered)
    df = pd.concat([df, user_df]).reset_index(drop=True)
    df.drop_duplicates(inplace=True)

    if model == 'BPR':
        # Add likes to dataset
        likes = pd.read_csv('likes.csv', quotechar='"', escapechar="\\", on_bad_lines='skip', engine="python")
        likes['rating_val'] = -1
        df = pd.concat([df, likes[['movie_id', 'user_id', 'rating_val']]], ignore_index=True)
        df = df.drop_duplicates(subset=['movie_id', 'user_id'], ignore_index=True)

        # Filter movies with few interactions
        movie_threshold = 5
        df = df[df['movie_id'].isin(df.groupby('movie_id').size()[lambda x: x >= movie_threshold].index)]

        user_mapping = {id: index for index, id in enumerate(df['user_id'].unique())}
        movie_mapping = {id: index for index, id in enumerate(df['movie_id'].unique())}
        reverse_movie_mapping = {index: id for id, index in movie_mapping.items()}

        df['user_idx'] = df['user_id'].map(user_mapping)
        df['movie_idx'] = df['movie_id'].map(movie_mapping)
        df['rating_val'] = 1  # Set all interactions to 1 for implicit feedback

        sparse_matrix = csr_matrix(
            (df['rating_val'], (df['user_idx'], df['movie_idx'])),
            shape=(df['user_idx'].max() + 1, df['movie_idx'].max() + 1)
        )

        algo = BayesianPersonalizedRanking(factors=num_factors, learning_rate=learning_rate,
                                           regularization=regularization, random_state=42)
        algo.fit(sparse_matrix.T)

        bpr_data = (user_mapping, movie_mapping, reverse_movie_mapping, sparse_matrix)

    else:
        # Surprise model fallback for SVD or NMF
        from surprise import SVD, NMF, Dataset, Reader
        reader = Reader(rating_scale=(1, 10))
        data = Dataset.load_from_df(df[["user_id", "movie_id", "rating_val"]], reader)
        algo = NMF(random_state=42) if model == 'NMF' else SVD(random_state=42)
        trainingSet = data.build_full_trainset()
        algo.fit(trainingSet)
        bpr_data = None

    user_watched_list = [x['movie_id'] for x in user_data_filtered]

    return algo, user_watched_list, bpr_data

def get_page_count(username):
    url = "https://letterboxd.com/{}/films/by/date"
    r = requests.get(url.format(username))

    soup = BeautifulSoup(r.text, "lxml")

    body = soup.find("body")

    try:
        if "error" in body["class"]:
            return -1, None
    except KeyError:
        print(body)
        return -1, None

    try:
        page_link = soup.findAll("li", attrs={"class", "paginate-page"})[-1]
        num_pages = int(page_link.find("a").text.replace(",", ""))
        display_name = (
            body.find("section", attrs={"class": "profile-header"})
            .find("h1", attrs={"class": "title-3"})
            .text.strip()
        )
    except IndexError:
        num_pages = 1
        display_name = None

    return num_pages, display_name

async def get_user_ratings(
    username,
    db_cursor=None,
    mongo_db=None,
    store_in_db=True,
    num_pages=None,
    return_unrated=False,
):
    url = "https://letterboxd.com/{}/films/by/date/page/{}/"

    if not num_pages:
        user = db_cursor.find_one({"username": username})
        num_pages = user["recent_page_count"]

    async with ClientSession() as session:
        tasks = [
            asyncio.ensure_future(
                fetch(url.format(username, i + 1), session, {"username": username})
            )
            for i in range(num_pages)
        ]
        scrape_responses = await asyncio.gather(*tasks)
        scrape_responses = [x for x in scrape_responses if x]

    tasks = [
        asyncio.ensure_future(
            generate_ratings_operations(
                response, send_to_db=store_in_db, return_unrated=return_unrated
            )
        )
        for response in scrape_responses
    ]
    parse_responses = await asyncio.gather(*tasks)

    if not store_in_db:
        # Flatten the raw data into a single list
        parse_responses = list(
            chain.from_iterable(list(chain.from_iterable(parse_responses)))
        )
        return parse_responses

    upsert_ratings_operations = []
    upsert_movies_operations = []
    for response in parse_responses:
        upsert_ratings_operations += response[0]
        upsert_movies_operations += response[1]

    return upsert_ratings_operations, upsert_movies_operations

async def get_user_data(username, data_opt_in=False):
    num_pages, display_name = get_page_count(username)

    if num_pages == -1:
        return [], "user_not_found"

    user_ratings = await get_user_ratings(
        username,
        db_cursor=None,
        mongo_db=None,
        store_in_db=False,  # Ensure we get raw data
        num_pages=num_pages,
        return_unrated=True,
    )

    # Filter out items where no rating or like is present
    user_ratings = [x for x in user_ratings if x["rating_val"] >= 0 or x["rating_val"] == -1]

    if data_opt_in:
        send_to_db(username, display_name, user_ratings=user_ratings)

    return user_ratings, "success"

df = training_df.copy()

"""
user_data, status = await get_user_data('geraldne')

if status == "success":
    user_data_train, user_data_test = train_test_split(
                user_data, test_size=0.2, random_state=42, stratify=[val == -1 for val in rating_vals])
    algo, user_watched_list, bpr_data = build_model(df, user_data_train, model='BPR')
"""

'\nuser_data, status = await get_user_data(\'geraldne\')\n\nif status == "success":\n    user_data_train, user_data_test = train_test_split(\n                user_data, test_size=0.2, random_state=42, stratify=[val == -1 for val in rating_vals])\n    algo, user_watched_list, bpr_data = build_model(df, user_data_train, model=\'BPR\')\n'

In [ ]:
from collections import defaultdict

try:
    from .db_config import config
except ImportError:
    config = None


def get_top_n(predictions, n=20):
    top_n = [(iid, est) for uid, iid, true_r, est, _ in predictions]
    top_n.sort(key=lambda x: (x[1], random.random()), reverse=True)

    return top_n[:n]


def run_model(username, algo, user_watched_list, user_data_test, threshold_movie_list, bpr_data=None, num_recommendations=100):

    user_watched_test = [(x['movie_id'], x['rating_val']) for x in user_data_test]

    unwatched_movies = [x for x in threshold_movie_list if x not in user_watched_list]

    if bpr_data:
        user_mapping, movie_mapping, reverse_movie_mapping, sparse_matrix = bpr_data
        if username in user_mapping:
            user_idx = user_mapping[username]
            if 0 <= user_idx < sparse_matrix.shape[0]:
                user_items = sparse_matrix[user_idx]
                recommendations = algo.recommend(user_idx, user_items, N=num_recommendations, filter_already_liked_items=False)
                item_indices, scores = recommendations
                top_n = [(reverse_movie_mapping[item_idx], score) for item_idx, score in zip(item_indices, scores) if item_idx in reverse_movie_mapping]
                top_n = sorted(top_n, key=lambda x: x[1], reverse=True)
            else:
                raise IndexError(f"user_idx {user_idx} out of bounds for sparse_matrix with shape {sparse_matrix.shape}.")
        else:
            raise ValueError(f"username '{username}' not found in user_mapping.")
    else:
        predictions = algo.test([(username, x, 0) for x in unwatched_movies])
        top_n = get_top_n(predictions, num_recommendations)

    movie_fields = ["image_url", "movie_id", "movie_title", "year_released", "genres", "original_language", "popularity", "runtime", "release_date"]
    metrics = compute_metrics(top_n, user_watched_test)
    movie_ids = [x[0] for x in top_n]
    filtered_movies = movies[movies["movie_id"].isin(movie_ids)]
    movie_data = {
        row["movie_id"]: {k: row[k] for k in filtered_movies.columns if k in movie_fields}
        for _, row in filtered_movies.iterrows()
    }

    return_object = [
        {"movie_id": x[0], "predicted_rating": round(x[1], 3), "movie_data": movie_data[x[0]]}
        for x in top_n if x[0] in movie_data.keys()
    ]
    return return_object, metrics

#recs, metrics = run_model('geraldne', algo, user_watched_list, user_data_test, threshold_movie_list, bpr_data, 10)

In [ ]:
async def fetch_all_user_data(users_to_test, get_user_data):
    """
    Fetch data for all users asynchronously and store it in a dictionary.

    Parameters:
        users_to_test (list): List of usernames to fetch data for.
        get_user_data (function): Asynchronous function to fetch user data.

    Returns:
        user_data_dict (dict): Dictionary with username as key and user data as value.
    """
    user_data_dict = {}

    async def fetch_user(username):
        """Fetch data for a single user."""
        print(f"Fetching data for user: {username}")
        user_data, status = await get_user_data(username)
        if status and user_data:
            user_data_dict[username] = user_data
        else:
            print(f"Failed to fetch data for user: {username}. Skipping.")

    # Create a list of tasks to fetch data for all users concurrently
    tasks = [fetch_user(username) for username in users_to_test]
    await asyncio.gather(*tasks)

    return user_data_dict

def save_user_data(user_data_dict, filename):
    with open(filename, 'wb') as f:
        pickle.dump(user_data_dict, f)
    print(f"User data saved to {filename}")

def load_user_data(filename):
    with open(filename, 'rb') as f:
        user_data_dict = pickle.load(f)
    print(f"User data loaded from {filename}")
    return user_data_dict

from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

def evaluate_models(users_to_test, user_data_dict, df, threshold_movie_list):
    results = []
    recommendations = {}

    def process_user(username):
        user_results = []
        user_recommendations = {}

        if username not in user_data_dict:
            print(f"No data available for user: {username}. Skipping.")
            return None

        try:
            # Extract user-specific data and split into train/test
            user_data = user_data_dict[username]
            rating_vals = [x['rating_val'] for x in user_data]
            user_data_train, user_data_test = train_test_split(
                user_data, test_size=0.2, random_state=42, stratify=[val == -1 for val in rating_vals]
            )
            user_watched_test = [(x['movie_id'], x['rating_val']) for x in user_data_test]

            # Initialize the user's entry in recommendations
            user_recommendations = {}

        except Exception as e:
            print(f"Error splitting data for user '{username}': {e}")
            return None

        # Train and evaluate each model
        for model_name in ['SVD', 'NMF', 'BPR']:
            try:
                # Build the model and retrieve necessary data
                algo, user_watched_list, bpr_data = build_model(df, user_data_train, model=model_name)

                # Run the model and retrieve recommendations and metrics
                recs, metrics = run_model(
                    username,
                    algo,
                    user_watched_list,
                    user_data_test,
                    threshold_movie_list,
                    bpr_data,
                    num_recommendations=50
                )

                # Add user and model information to metrics
                metrics['model'] = model_name
                metrics['user'] = username
                user_results.append(metrics)

                # Store recommendations
                user_recommendations[model_name] = recs

            except ValueError as ve:
                print(f"Skipping user '{username}' for model '{model_name}' due to ValueError: {ve}")
            except Exception as e:
                print(f"Unexpected error for user '{username}' and model '{model_name}': {e}")

        return user_results, user_recommendations

    # Parallelize user processing
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_user, username): username for username in users_to_test}

        for future in concurrent.futures.as_completed(futures):
            username = futures[future]
            try:
                user_results, user_recommendations = future.result()
                if user_results:
                    results.extend(user_results)
                if user_recommendations:
                    recommendations[username] = user_recommendations
            except Exception as e:
                print(f"Error processing user {username}: {e}")

    metrics_df = pd.DataFrame(results)
    return metrics_df, recommendations

async def main(users_to_test, df, get_user_data, threshold_movie_list, data_file=None):
    """
    Main function to fetch data, save/load it, and evaluate models.

    Parameters:
        users_to_test (list): List of usernames to evaluate.
        df (pd.DataFrame): Ratings DataFrame.
        get_user_data (function): Asynchronous function to fetch user data.
        threshold_movie_list (list): List of movies to recommend from.
        data_file (str): Path to save/load user data.

    Returns:
        metrics_df (pd.DataFrame): DataFrame containing metrics for each model and user.
        recommendations (dict): Nested dictionary with recommendations for each user and model.
    """
    # Check if data_file exists
    if data_file:
        try:
            # Try loading user data from the file
            user_data_dict = load_user_data(data_file)
        except FileNotFoundError:
            # If file not found, fetch and save data
            print(f"{data_file} not found. Fetching user data.")
            user_data_dict = await fetch_all_user_data(users_to_test, get_user_data)
            save_user_data(user_data_dict, data_file)
    else:
        # If no file provided, fetch user data
        user_data_dict = await fetch_all_user_data(users_to_test, get_user_data)

    print(f"Fetched data for {len(user_data_dict)} users.")

    # Evaluate models using the fetched data
    metrics_df, recommendations = evaluate_models(users_to_test, user_data_dict, df, threshold_movie_list)
    return metrics_df, recommendations

# List of users you want to evaluate
usernames = pd.read_csv('sample_data.csv')
users_to_test = list(usernames['user_id'].unique())[0]

# File to save/load user data
data_file = 'user_data_test_2.pkl'

# Run the main function
metrics_df, recommendations = asyncio.run(main(users_to_test, df, get_user_data, threshold_movie_list, data_file=data_file))

User data loaded from user_data_test_2.pkl
Fetched data for 6 users.


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.025688648223876953 seconds
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.006682634353637695 seconds
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.023427963256835938 seconds
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.007912158966064453 seconds
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.013580083847045898 seconds
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
#users_to_test

['ev1lcatboy', 'dyketactics', 'lanmar', 'antovolk', 'genyasapphic', 'hashbro']

In [ ]:
summary = metrics_df.groupby('model').mean(numeric_only=True)
print(summary)

       precision_at_k  mean_reciprocal_rank      ndcg
model                                                
BPR              0.05              0.187801  0.031730
NMF              0.00              0.018568  0.005717
SVD              0.00              0.012821  0.004692


In [ ]:
with open("recommendations.pkl", "wb") as file:
    pickle.dump(recommendations, file)

In [ ]:
metrics_df.to_csv('metrics_results_ranked.csv')

In [ ]:
# load the data
reviews_df = pd.read_csv('reviews.csv')
reviews_df = reviews_df.head(12000)
extra = pd.read_csv('sample_data.csv')
og_users = reviews_df['user_id'].unique()
extra_users = extra['user_id'].unique()
reviews_df = pd.concat([reviews_df, extra])
reviews_df

,_id,movie_id,user_id,rating_val,review_text
0,ObjectId(673bc832de1ba8599d492f03),emilia-perez,romankirby,7.0,65Some of Emilia Pérez works so wonderfully an...
1,ObjectId(673bc832de1ba8599d492f04),saturday-night-2024,romankirby,8.0,71First time in a while I’ve seen a film that ...
2,ObjectId(673bc832de1ba8599d492f05),chimp-crazy,romankirby,7.0,68Absolutely BANANAS!!
3,ObjectId(673bc832de1ba8599d492f06),chompy-the-girls,romankirby,3.0,26Chompy’s head game must be crazy.
4,ObjectId(673bc832de1ba8599d492f07),pompeii,romankirby,1.0,"7If the real Pompeii was this boring, I’m glad..."
...,...,...,...,...,...
2434,NaN,wolfs,laurenwilliams,8.0,Just accidentally watched my first Christmas f...
2435,NaN,paddington-in-peru,laurenwilliams,10.0,This review may contain spoilers. I can handle...
2436,NaN,juror-2,laurenwilliams,8.0,What a predicament.
2437,NaN,the-shawshank-redemption,laurenwilliams,10.0,It’s really baffling when people say plot does...


In [ ]:
# map from rating_val to sentiment
reviews_df['sentiment'] = reviews_df['rating_val']
reviews_df

,_id,movie_id,user_id,rating_val,review_text,sentiment
0,ObjectId(673bc832de1ba8599d492f03),emilia-perez,romankirby,7.0,65Some of Emilia Pérez works so wonderfully an...,7.0
1,ObjectId(673bc832de1ba8599d492f04),saturday-night-2024,romankirby,8.0,71First time in a while I’ve seen a film that ...,8.0
2,ObjectId(673bc832de1ba8599d492f05),chimp-crazy,romankirby,7.0,68Absolutely BANANAS!!,7.0
3,ObjectId(673bc832de1ba8599d492f06),chompy-the-girls,romankirby,3.0,26Chompy’s head game must be crazy.,3.0
4,ObjectId(673bc832de1ba8599d492f07),pompeii,romankirby,1.0,"7If the real Pompeii was this boring, I’m glad...",1.0
...,...,...,...,...,...,...
2434,NaN,wolfs,laurenwilliams,8.0,Just accidentally watched my first Christmas f...,8.0
2435,NaN,paddington-in-peru,laurenwilliams,10.0,This review may contain spoilers. I can handle...,10.0
2436,NaN,juror-2,laurenwilliams,8.0,What a predicament.,8.0
2437,NaN,the-shawshank-redemption,laurenwilliams,10.0,It’s really baffling when people say plot does...,10.0


In [ ]:
# tokenization
reviews_df['tokens'] = [simple_preprocess(line, deacc=True) for line in reviews_df['review_text']]
reviews_df['tokens']

,tokens
0,"[some, of, emilia, perez, works, so, wonderful..."
1,"[first, time, in, while, ve, seen, film, that,..."
2,"[absolutely, bananas]"
3,"[chompy, head, game, must, be, crazy]"
4,"[if, the, real, pompeii, was, this, boring, gl..."
...,...
2434,"[just, accidentally, watched, my, first, chris..."
2435,"[this, review, may, contain, spoilers, can, ha..."
2436,"[what, predicament]"
2437,"[it, really, baffling, when, people, say, plot..."


In [ ]:
# stemming porterstemmer
porter_stemmer = PorterStemmer()
reviews_df['porter_stemmed'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in reviews_df['tokens']]
reviews_df['porter_stemmed']

,porter_stemmed
0,"[some, of, emilia, perez, work, so, wonderfull..."
1,"[first, time, in, while, ve, seen, film, that,..."
2,"[absolut, banana]"
3,"[chompi, head, game, must, be, crazi]"
4,"[if, the, real, pompeii, wa, thi, bore, glad, ..."
...,...
2434,"[just, accident, watch, my, first, christma, f..."
2435,"[thi, review, mai, contain, spoiler, can, hand..."
2436,"[what, predica]"
2437,"[it, realli, baffl, when, peopl, sai, plot, do..."


In [ ]:
# stemming lancasterstemmer
lancaster_stemmer = LancasterStemmer()
reviews_df['lancaster_stemmed'] = [[lancaster_stemmer.stem(word) for word in tokens] for tokens in reviews_df['tokens']]
reviews_df['lancaster_stemmed']

,lancaster_stemmed
0,"[som, of, emil, perez, work, so, wond, and, th..."
1,"[first, tim, in, whil, ve, seen, film, that, f..."
2,"[absolv, banana]"
3,"[chompy, head, gam, must, be, crazy]"
4,"[if, the, real, pompei, was, thi, bor, glad, t..."
...,...
2434,"[just, accid, watch, my, first, christmas, fil..."
2435,"[thi, review, may, contain, spoil, can, handl,..."
2436,"[what, predica]"
2437,"[it, real, baffl, when, peopl, say, plot, does..."


In [ ]:
# stemming snowballstemmer
snowball_stemmer = SnowballStemmer(language='english')
reviews_df['snowball_stemmed'] = [[snowball_stemmer.stem(word) for word in tokens] for tokens in reviews_df['tokens']]
reviews_df['snowball_stemmed']

,snowball_stemmed
0,"[some, of, emilia, perez, work, so, wonder, an..."
1,"[first, time, in, while, ve, seen, film, that,..."
2,"[absolut, banana]"
3,"[chompi, head, game, must, be, crazi]"
4,"[if, the, real, pompeii, was, this, bore, glad..."
...,...
2434,"[just, accident, watch, my, first, christma, f..."
2435,"[this, review, may, contain, spoiler, can, han..."
2436,"[what, predica]"
2437,"[it, realli, baffl, when, peopl, say, plot, do..."


In [ ]:
reviews_df

,_id,movie_id,user_id,rating_val,review_text,sentiment,tokens,porter_stemmed,lancaster_stemmed,snowball_stemmed
0,ObjectId(673bc832de1ba8599d492f03),emilia-perez,romankirby,7.0,65Some of Emilia Pérez works so wonderfully an...,7.0,"[some, of, emilia, perez, works, so, wonderful...","[some, of, emilia, perez, work, so, wonderfull...","[som, of, emil, perez, work, so, wond, and, th...","[some, of, emilia, perez, work, so, wonder, an..."
1,ObjectId(673bc832de1ba8599d492f04),saturday-night-2024,romankirby,8.0,71First time in a while I’ve seen a film that ...,8.0,"[first, time, in, while, ve, seen, film, that,...","[first, time, in, while, ve, seen, film, that,...","[first, tim, in, whil, ve, seen, film, that, f...","[first, time, in, while, ve, seen, film, that,..."
2,ObjectId(673bc832de1ba8599d492f05),chimp-crazy,romankirby,7.0,68Absolutely BANANAS!!,7.0,"[absolutely, bananas]","[absolut, banana]","[absolv, banana]","[absolut, banana]"
3,ObjectId(673bc832de1ba8599d492f06),chompy-the-girls,romankirby,3.0,26Chompy’s head game must be crazy.,3.0,"[chompy, head, game, must, be, crazy]","[chompi, head, game, must, be, crazi]","[chompy, head, gam, must, be, crazy]","[chompi, head, game, must, be, crazi]"
4,ObjectId(673bc832de1ba8599d492f07),pompeii,romankirby,1.0,"7If the real Pompeii was this boring, I’m glad...",1.0,"[if, the, real, pompeii, was, this, boring, gl...","[if, the, real, pompeii, wa, thi, bore, glad, ...","[if, the, real, pompei, was, thi, bor, glad, t...","[if, the, real, pompeii, was, this, bore, glad..."
...,...,...,...,...,...,...,...,...,...,...
2434,NaN,wolfs,laurenwilliams,8.0,Just accidentally watched my first Christmas f...,8.0,"[just, accidentally, watched, my, first, chris...","[just, accident, watch, my, first, christma, f...","[just, accid, watch, my, first, christmas, fil...","[just, accident, watch, my, first, christma, f..."
2435,NaN,paddington-in-peru,laurenwilliams,10.0,This review may contain spoilers. I can handle...,10.0,"[this, review, may, contain, spoilers, can, ha...","[thi, review, mai, contain, spoiler, can, hand...","[thi, review, may, contain, spoil, can, handl,...","[this, review, may, contain, spoiler, can, han..."
2436,NaN,juror-2,laurenwilliams,8.0,What a predicament.,8.0,"[what, predicament]","[what, predica]","[what, predica]","[what, predica]"
2437,NaN,the-shawshank-redemption,laurenwilliams,10.0,It’s really baffling when people say plot does...,10.0,"[it, really, baffling, when, people, say, plot...","[it, realli, baffl, when, peopl, sai, plot, do...","[it, real, baffl, when, peopl, say, plot, does...","[it, realli, baffl, when, peopl, say, plot, do..."


In [ ]:
# building dictionary
mydict = corpora.Dictionary(reviews_df['snowball_stemmed'])
print('total unique words:')
print(len(mydict.token2id))
print('\nsample data from dictionary:')
i = 0
for key in mydict.token2id.keys():
  print('word: {} - id: {} '.format(key, mydict.token2id[key]))
  if i == 3:
    break
  i += 1

total unique words:
29000

sample data from dictionary:
word: admir - id: 0 
word: also - id: 1 
word: and - id: 2 
word: at - id: 3 


In [ ]:
reviews_df.dropna(subset=['sentiment'], inplace=True)

In [ ]:
# split into train (~70%) and test (~30%) sets
extra_users = reviews_df[reviews_df['user_id'].isin(extra_users)]
balanced_df = reviews_df[reviews_df['user_id'].isin(og_users)]
x_train, x_test, y_train, y_test = train_test_split(balanced_df[['movie_id','user_id','rating_val','review_text','tokens','snowball_stemmed']],
                                                    balanced_df['sentiment'],
                                                    shuffle=True,
                                                    test_size=.3,
                                                    random_state=15,
                                                    stratify=balanced_df['sentiment'])
print('value counts for train sentiments')
print(y_train.value_counts())
print('value counts for test sentiments')
print(y_test.value_counts())
x_train = x_train.reset_index()
x_test = x_test.reset_index()
y_train = y_train.to_frame()
y_train = y_train.reset_index()
y_test = y_test.to_frame()
y_test = y_test.reset_index()
x_train = pd.concat([x_train, extra_users])
y_train = pd.concat([y_train, extra_users['sentiment']])

value counts for train sentiments
sentiment
8.0     2119
6.0     1198
7.0     1137
10.0     909
9.0      835
4.0      724
5.0      393
2.0      309
3.0      171
1.0       78
Name: count, dtype: int64
value counts for test sentiments
sentiment
8.0     908
6.0     514
7.0     488
10.0    389
9.0     358
4.0     310
5.0     168
2.0     133
3.0      73
1.0      34
Name: count, dtype: int64


In [ ]:
# create tfidf model
corpus = [mydict.doc2bow(line) for line in balanced_df['snowball_stemmed']]
tfidf_model = TfidfModel(corpus)

In [ ]:
# generate tfidf vectors
start = time.time()
tfidf_filename = 'train_review_tfidf.csv'
vocab_len = len(mydict.token2id)
with open(tfidf_filename, 'w+') as tfidf_file:
  for index, row in x_train.iterrows():
    doc = mydict.doc2bow(row['snowball_stemmed'])
    features = gensim.matutils.corpus2csc([tfidf_model[doc]], num_terms=vocab_len).toarray()[:,0]
    if index == 0:
      header = ','.join(str(mydict[ele]) for ele in range(vocab_len))
      print(header)
      print(tfidf_model[doc])
      tfidf_file.write(header)
      tfidf_file.write('\n')
    line1 = ','.join( [str(vector_element) for vector_element in features])
    tfidf_file.write(line1)
    tfidf_file.write('\n')
  print('time taken to create tfidf for :' + str(time.time() - start))

admir,also,and,at,back,bright,but,can,cereal,emilia,even,exist,fail,film,fuck,good,hard,how,imagin,in,it,like,of,onli,other,part,past,perez,put,realli,shine,sit,so,some,the,this,to,wallpap,well,wonder,work,your,all,almost,as,been,element,everi,feel,first,fun,have,love,make,probabl,seen,should,singl,take,that,time,ve,was,watch,while,absolut,banana,be,chompi,crazi,game,head,must,bore,glad,got,if,pompeii,real,shit,smoke,cast,execut,great,kill,plot,stellar,derogatori,one,weirdest,anna,behind,camera,debut,didn,directori,disappoint,for,front,hour,incred,is,kendrick,sharp,she,with,woman,bottl,go,harder,vicodin,voss,would,an,dumb,guess,improv,know,think,appreci,argument,cash,could,differ,do,flaw,grab,has,interest,just,least,made,money,much,not,noth,or,pointless,relat,say,slow,someth,still,they,tri,won,art,guy,nice,start,veri,anyth,begin,danc,doesn,down,enjoy,ever,give,gnarliest,howev,may,open,rest,scene,unfortun,cook,dop,editor,on,thought,about,contain,durden,fr,handl,he,liter,me,narrat,now,re

In [ ]:
# train sentiment classification model using tfidf vectors
start_time = time.time()
tfidf_df = pd.read_csv('train_review_tfidf.csv')
clf_decision_tfidf = DecisionTreeClassifier(random_state=2, class_weight='balanced')
clf_decision_tfidf.fit(tfidf_df, y_train['sentiment'])
print('time taken to fit the tfidf as input for classifier: ' + str(time.time() - start_time))

time taken to fit the tfidf as input for classifier: 182.21785831451416


In [ ]:
# print important features
importances = list(clf_decision_tfidf.feature_importances_)
feature_importances = [(feature, round(importance, 10)) for feature, importance in zip(tfidf_df.columns, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
top_i = 0
for pair in feature_importances:
  print('variable: {:20} importance: {}'.format(*pair))
  if top_i == 20:
    break
  top_i += 1

variable: the                  importance: 0.0232368836
variable: and                  importance: 0.0209983847
variable: of                   importance: 0.0206634835
variable: this                 importance: 0.018082774
variable: to                   importance: 0.0162553828
variable: concern              importance: 0.0160940307
variable: in                   importance: 0.0159739893
variable: is                   importance: 0.015823505
variable: it                   importance: 0.0140556591
variable: that                 importance: 0.0132392918
variable: unconvinc            importance: 0.0127756955
variable: film                 importance: 0.0103606474
variable: but                  importance: 0.0102698337
variable: movi                 importance: 0.008749427
variable: good                 importance: 0.0085898237
variable: star                 importance: 0.0084214288
variable: with                 importance: 0.0083775793
variable: like                 importance: 0.007376

In [ ]:
# test model
test_features_tfidf = []
start_time = time.time()
for index, row in x_test.iterrows():
  doc = mydict.doc2bow(row['snowball_stemmed'])
  features = gensim.matutils.corpus2csc([tfidf_model[doc]], num_terms=vocab_len).toarray()[:,0]
  test_features_tfidf.append(features)
test_predictions_tfidf = clf_decision_tfidf.predict(test_features_tfidf)
print(classification_report(y_test['sentiment'], test_predictions_tfidf))
print('time taken to predict using tfidf:' + str(time.time() - start_time))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

         1.0       0.05      0.15      0.07        34
         2.0       0.26      0.23      0.24       133
         3.0       0.05      0.04      0.04        73
         4.0       0.33      0.34      0.34       310
         5.0       0.10      0.10      0.10       168
         6.0       0.26      0.28      0.27       514
         7.0       0.17      0.20      0.19       488
         8.0       0.34      0.29      0.31       908
         9.0       0.15      0.14      0.15       358
        10.0       0.15      0.14      0.15       389

    accuracy                           0.23      3375
   macro avg       0.19      0.19      0.19      3375
weighted avg       0.24      0.23      0.23      3375

time taken to predict using tfidf:3.3446950912475586


In [ ]:
def get_sentiment(df):
  test_features_tfidf = []
  for index, row in df.iterrows():
    doc = mydict.doc2bow(row['snowball_stemmed'])
    features = gensim.matutils.corpus2csc([tfidf_model[doc]], num_terms=vocab_len).toarray()[:,0]
    test_features_tfidf.append(features)
  return clf_decision_tfidf.predict(test_features_tfidf)

In [ ]:
def pr_sent(u_a, i_j, similar_users, sim):
  user_ratings = x_train[x_train['user_id'] == u_a]
  user_ratings['sentiment_pred'] = get_sentiment(user_ratings)
  u_a_avg_rating = np.mean(user_ratings['sentiment_pred'])
  num = 0
  denom = 0
  for i in range(len(similar_users)):
    similar_user_ratings = x_train[x_train['user_id'] == similar_users[i]]
    similar_user_ratings['sentiment_pred'] = get_sentiment(similar_user_ratings)
    u_i_avg_rating = np.mean(similar_user_ratings['sentiment_pred'])
    similarity = sim[i]
    r_ij = similar_user_ratings.loc[similar_user_ratings['movie_id'] == i_j, 'sentiment_pred'].values
    num += similarity * (r_ij - u_i_avg_rating)
    denom += abs(similarity)
  pr = u_a_avg_rating + (num / denom)
  return pr

In [ ]:
def sentiment_rating_pred(user, item):
  user_ratings = x_train[x_train['user_id'] == user]
  user_ratings['sentiment_pred'] = get_sentiment(user_ratings)
  movies = []
  users = []
  for movie in x_train['movie_id'].values:
    if movie in user_ratings['movie_id'].values:
      movies.append(movie)
  for user_b in x_train['user_id'].values:
    user_b_ratings = x_train[x_train['user_id'] == user_b]
    for movie in movies:
      if (item in user_b_ratings['movie_id'].values) and (movie in user_b_ratings['movie_id'].values):
        users.append(user_b)
  similar_users = list(set(users) - {user})
  sim = []
  if len(similar_users) > 0:
    for similar_user in similar_users:
      similar_user_ratings = x_train[x_train['user_id'] == similar_user]
      similar_user_ratings['sentiment_pred'] = get_sentiment(similar_user_ratings)
      common_movies = pd.merge(user_ratings, similar_user_ratings, on='movie_id', how='inner')
      user_vec = common_movies[common_movies['user_id_x'] == user]['sentiment_pred_x']
      similar_user_vec = common_movies[common_movies['user_id_y'] == similar_user]['sentiment_pred_y']
      if user_vec.empty or similar_user_vec.empty:
        continue
      s = cosine_similarity(user_vec.values.reshape(1, -1), similar_user_vec.values.reshape(1, -1))
      sim.append(s)
    return pr_sent(user, item, similar_users, sim)
  else:
    return np.mean(user_ratings['sentiment_pred'])

In [ ]:
def new_rating(beta, pr_mf, pr_sentiment):
  return beta * pr_mf + (1 - beta) * pr_sentiment

In [ ]:
# combine models
betas = [.5]
users = ['ev1lcatboy']
for beta in betas:
  updated_recs = recommendations['ev1lcatboy'].copy()
  for user in users:
    for model in recommendations[user]:
      if model == 'SVD':
        i = 0
        for item in recommendations[user][model]:
          pr_mf = recommendations[user][model][i]['predicted_rating']
          pr_sentiment = sentiment_rating_pred(user, item['movie_id'])
          updated_rating = new_rating(beta, pr_mf, pr_sentiment)
          updated_recs[model][i]['predicted_rating'] = updated_rating
          i += 1
  with open("recommendations-sentiment.pkl", "wb") as file:
    pickle.dump(updated_recs, file)

0.5
ev1lcatboy
SVD
the-farmers-wife
0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
<ipython-input-78-b3c2f09cd182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings['sentiment_pred'] = get_sentiment(user_ratings)


the-tales-of-hoffmann
1


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
<ipython-input-78-b3c2f09cd182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings['sentiment_pred'] = get_sentiment(user_ratings)


miss-representation
2


...

castle-of-blood
48


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
<ipython-input-78-b3c2f09cd182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings['sentiment_pred'] = get_sentiment(user_ratings)


autumn-sonata
49


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
<ipython-input-78-b3c2f09cd182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings['sentiment_pred'] = get_sentiment(user_ratings)


NMF
BPR


In [ ]:
updated_recs['SVD'][0]['predicted_rating'] = updated_recs['SVD'][0]['predicted_rating'][0][0]

{'SVD': [{'movie_id': 'the-farmers-wife',
   'predicted_rating': 7.117852605633803,
   'movie_data': {'movie_id': 'the-farmers-wife',
    'image_url': 'film-poster/2/8/3/8/5/28385-the-farmer-s-wife-0-230-0-345-crop',
    'movie_title': "The Farmer's Wife",
    'year_released': 1928.0}},
  {'movie_id': 'the-tales-of-hoffmann',
   'predicted_rating': 8.5625,
   'movie_data': {'movie_id': 'the-tales-of-hoffmann',
    'image_url': 'film-poster/2/0/8/4/1/20841-the-tales-of-hoffmann-0-230-0-345-crop',
    'movie_title': 'The Tales of Hoffmann',
    'year_released': 1951.0}},
  {'movie_id': 'miss-representation',
   'predicted_rating': 8.5625,
   'movie_data': {'movie_id': 'miss-representation',
    'image_url': 'film-poster/6/4/2/4/8/64248-miss-representation-0-230-0-345-crop',
    'movie_title': 'Miss Representation',
    'year_released': 2011.0}},
  {'movie_id': 'blue-collar',
   'predicted_rating': 8.5625,
   'movie_data': {'movie_id': 'blue-collar',
    'image_url': 'film-poster/4/2/7/7/

In [ ]:
'''
TO TEST MULTIPLE BETAS, USERS, AND MODELS:
betas = [.3, .5, .7]
for beta in betas:
  updated_recs = recommendations.copy()
  for user in recommendations:
    for model in recommendations[user]:
      if model != 'BPR':
        i = 0
        for item in recommendations[user][model]:
          pr_mf = recommendations[user][model][i]['predicted_rating']
          pr_sentiment = sentiment_rating_pred(user, item['movie_id'])
          updated_rating = new_rating(beta, pr_mf, pr_sentiment)
          updated_recs[user][model][i]['predicted_rating'] = updated_rating
          i += 1
  with open("recommendations-sentiment-beta=" + str(beta) + '.pkl', "wb") as file:
    pickle.dump(updated_recs, file)
'''

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
<ipython-input-78-b3c2f09cd182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings['sentiment_pred'] = get_sentiment(user_ratings)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
<ipython-input-78-b3c2f09cd182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

KeyboardInterrupt: 

In [ ]:
# getting metrics of combined model
users_to_test = ['ev1lcatboy']
user_data_dict = {}
with open('user_data_test_2.pkl', 'rb') as f:
    user_data_dict = pickle.load(f)
user_data = user_data_dict['ev1lcatboy']
rating_vals = [x['rating_val'] for x in user_data]
user_data_train, user_data_test = train_test_split(
                user_data, test_size=0.2, random_state=42, stratify=[val == -1 for val in rating_vals])
evaluate_models(users_to_test, user_data_dict, training_df, threshold_movie_list)
recs = []
test = []
for model in updated_recs:
  for item in updated_recs[model]:
    recs.append((item['movie_id'], item['predicted_rating']))
for item in user_data_test:
  if item['rating_val'] != -1:
    test.append((item['movie_id'], item['rating_val']))
compute_metrics(recs, test, k=10)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0031321048736572266 seconds
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

{'precision_at_k': 0.0,
 'mean_reciprocal_rank': 0.008264462809917356,
 'ndcg': 0.0078927040800272}